In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install tensorflow-gpu==2.0.0-beta1

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [2]:
import tensorflow_datasets as tfds
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks")
os.listdir()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['functions',
 'configuration',
 'data',
 'Copy of imbalanced_data.ipynb',
 'Copy of Day1&2_feed_forward_NN_classification_helper.ipynb',
 'prediction.csv',
 'Day1&2_feed_forward_NN_regression_helper.ipynb',
 'day3-4',
 'Day3+4_CNN_COPY',
 'Untitled0.ipynb',
 '.ipynb_checkpoints',
 'Grad_CAM.ipynb',
 'quotes.json',
 'training_checkpoints',
 'training_checkpoints_lstm',
 'Copy of text_generation.ipynb',
 'text_generation.ipynb',
 'Copy of Day1&2_feed_forward_NN_regression.ipynb',
 'df_clean_newwer.pkl',
 'functions.py',
 '30_1_initial_modeling_investigation.ipynb']

In [0]:
import os
import re
import string
import itertools

import numpy as np
from numpy import mean
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn import model_selection


In [0]:
# own functions, placed in functions.py.
def make_reduced_df (df, machines_keep):
    ''' filters a data frame to contain the data for certain machines.
        input: df  = analysis data frame
               machines_keep = array of machines to keep
        output: df = filtered data frame
    '''
    df = df[df['machine'].isin(machines_keep)]
    return df

In [0]:
#print(os.getcwd())
#os.chdir('/Users/ooles/OneDrive/Desktop/Final_Project/spinningbytes')

In [0]:
os.getcwd()

'C:\\Users\\ooles\\OneDrive\\Desktop\\Final_Project\\spinningbytes'

In [5]:
df = pd.read_pickle("df_clean_newwer.pkl")
df.shape 


(365456, 10)

In [6]:
df['reference.text'].value_counts().unique()

array([16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2])

In [7]:
df['hypothesis.text'].value_counts().unique()

array([251, 215, 115,  79,  75,  74,  71,  65,  62,  57,  48,  47,  41,
        38,  33,  29,  28,  26,  24,  23,  22,  20,  19,  18,  17,  16,
        15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,
         2,   1])

In [8]:
df.head()

,identifier,speaker_id,file,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer,recomputed_wer
5,sw3112B-ms98-a-0063,1304,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,i i don't listen to in nearly enough it's the ...,i i don't listen to it nearly enough it's the ...,0.166667,0.166667
1341,sw3753A-ms98-a-0051,1477,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,you know i don't know if you saw it was it two...,you know i don't know if you saw it was a two ...,0.190476,0.190476
1339,sw3921B-ms98-a-0049,1406,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,and then the skin just basically scrapes right...,and then the skin just basically scrapes right...,0.055556,0.055556
1338,sw2763A-ms98-a-0010,1237,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,see i don't know what questions to ask because...,see i don't know what questions to ask because...,0.117647,0.117647
1337,sw3109B-ms98-a-0155,1023,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,they have a uh big you know it's a big complex...,they have a a big you know it's a big complexe...,0.040000,0.040000


In [9]:
df['reference.text']


5       i i don't listen to in nearly enough it's the ...
1341    you know i don't know if you saw it was it two...
1339    and then the skin just basically scrapes right...
1338    see i don't know what questions to ask because...
1337    they have a uh big you know it's a big complex...
                              ...                        
2547    but that that time is minuscule compared to wh...
2540    i enjoyed the book quite a bit too so that's t...
2535    uh up to a points things seemed to be working ...
2587    sure it's real easy to spend other people's mo...
3168    is is you know is good for both of them i know...
Name: reference.text, Length: 365456, dtype: object

In [0]:
#machines_keep = ['2', '3', '4', '5']
machines_keep = ['5']
df = make_reduced_df(df, machines_keep)

df2=df.copy()
#df = split_hypothesis (df)

In [11]:
df2.columns

Index(['identifier', 'speaker_id', 'file', 'corpus', 'configuration',
       'machine', 'reference.text', 'hypothesis.text', 'scoring.wer',
       'recomputed_wer'],
      dtype='object')

In [12]:
df2.head()

,identifier,speaker_id,file,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer,recomputed_wer
5,sw3112B-ms98-a-0063,1304,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,i i don't listen to in nearly enough it's the ...,i i don't listen to it nearly enough it's the ...,0.166667,0.166667
1341,sw3753A-ms98-a-0051,1477,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,you know i don't know if you saw it was it two...,you know i don't know if you saw it was a two ...,0.190476,0.190476
1339,sw3921B-ms98-a-0049,1406,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,and then the skin just basically scrapes right...,and then the skin just basically scrapes right...,0.055556,0.055556
1338,sw2763A-ms98-a-0010,1237,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,see i don't know what questions to ask because...,see i don't know what questions to ask because...,0.117647,0.117647
1337,sw3109B-ms98-a-0155,1023,switchboard_segmented__5_ae88.json,switchboard_segmented,5_ae88,5,they have a uh big you know it's a big complex...,they have a a big you know it's a big complexe...,0.040000,0.040000


In [0]:
#df2 = split_hypothesis (df2)

In [25]:
df2["reference.text"]

5       i i don't listen to in nearly enough it's the ...
1341    you know i don't know if you saw it was it two...
1339    and then the skin just basically scrapes right...
1338    see i don't know what questions to ask because...
1337    they have a uh big you know it's a big complex...
                              ...                        
424     in novels the hero has often pushed his meals ...
421            we moderns however see the absurdity of it
426     lady larkspur starts suddenly and turns toward...
427     then is the time to introduce a meal on the stage
425            enter hamlet with his favourite boar hound
Name: reference.text, Length: 46761, dtype: object

In [0]:
a = []
for text in df2["reference.text"]:
    a.append(tokenizer_en.encode(text))

In [0]:
tensor = tf.keras.preprocessing.sequence.pad_sequences(a,
                                                         padding='post')

In [21]:
tensor.shape

(46761, 192)

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (text for text in df2["reference.text"]), target_vocab_size=2**13)

In [16]:
sample_string = 'Transformer is awesome.'

tokenized_string = tokenizer_en.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer_en.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [8287, 1696, 7046, 3536, 171, 12, 3063, 1390, 8249]
The original string: Transformer is awesome.


In [17]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_en.decode([ts])))

8287 ----> T
1696 ----> ran
7046 ----> sf
3536 ----> orm
171 ----> er 
12 ----> is 
3063 ----> awe
1390 ----> some
8249 ----> .


In [28]:
tokenizer_en.vocab_size

8459

In [0]:
def tokenize(dataframe_column):
   
    a = []
    for text in dataframe_column:
        a.append(tokenizer_en.encode(text))
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(a,
                                                         padding='post')
    
    return tensor

In [29]:
tokenize(df2["reference.text"])

array([[   7,    7,   55, ...,    0,    0,    0],
       [   8,   30,    7, ...,    0,    0,    0],
       [   2,   53,    1, ...,    0,    0,    0],
       ...,
       [1348, 2316, 6230, ...,    0,    0,    0],
       [  53,   12,    1, ...,    0,    0,    0],
       [3004, 7628,   25, ...,    0,    0,    0]], dtype=int32)

In [30]:
tokenize(df2["hypothesis.text"])

array([[   7,    7,   55, ...,    0,    0,    0],
       [   8,   30,    7, ...,    0,    0,    0],
       [   2,   53,    1, ...,    0,    0,    0],
       ...,
       [1348, 2316,  381, ...,    0,    0,    0],
       [  53,   12,    1, ...,    0,    0,    0],
       [1076, 8235, 7628, ...,    0,    0,    0]], dtype=int32)

In [31]:
target_tensor = tokenize(df2['reference.text'])
target_tensor.shape

(46761, 192)

In [0]:
input_tensor = tokenize(df2['hypothesis.text'])

In [33]:
input_tensor.shape

(46761, 180)

In [34]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = model_selection.train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

37408 37408 9353 9353


In [0]:
#BUFFER_SIZE = 10000
#BATCH_SIZE = 64

In [0]:
#train_dataset = input_tensor_train.shuffle(BUFFER_SIZE)
#train_dataset = input_tensor_train.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

#test_dataset = input_tensor_val.padded_batch(BATCH_SIZE, input_tensor_val.output_shapes)


In [0]:
# Have a look at index to word mapping

def convert(text, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, text.index_word[t]))

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer_en.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(tensor.shape[1])
])


In [36]:
tensor.shape[1]

192

In [212]:
len(input_tensor_train)/57

656.280701754386

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [38]:
history = model.fit(x=input_tensor_train, y=target_tensor_train, epochs=10, batch_size=32,
                    validation_data=(input_tensor_val,target_tensor_val),
                    validation_steps=30)


Train on 37408 samples, validate on 9353 samples
Epoch 1/10
37408/37408 [==============================] - 37s 977us/sample - loss: -476.1345 - accuracy: 0.6624 - val_loss: -72.5357 - val_accuracy: 0.5990
Epoch 2/10
37408/37408 [==============================] - 29s 764us/sample - loss: -719.3064 - accuracy: 0.5933 - val_loss: -72.7240 - val_accuracy: 0.5938
Epoch 3/10
37408/37408 [==============================] - 28s 761us/sample - loss: -719.6881 - accuracy: 0.5922 - val_loss: -72.7233 - val_accuracy: 0.5938
Epoch 4/10
37408/37408 [==============================] - 28s 762us/sample - loss: -719.6881 - accuracy: 0.5924 - val_loss: -72.7155 - val_accuracy: 0.5938
Epoch 5/10
37408/37408 [==============================] - 28s 760us/sample - loss: -719.7235 - accuracy: 0.5923 - val_loss: -72.7088 - val_accuracy: 0.5938
Epoch 6/10
37408/37408 [==============================] - 28s 762us/sample - loss: -719.7519 - accuracy: 0.5920 - val_loss: -72.7085 - val_accuracy: 0.5938
Epoch 7/10
3740

In [0]:
out = model.predict(target_tensor[6])

In [40]:
Y = list()
for x in out.argmax(axis = 1)[:,None]:
    Y.append(x.sum())

Y = np.array(Y)
print(Y)

[43 43 49 49 49 49 49 49 43 49 49 43 49 49  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3]


In [41]:
out.argmax(axis = 1)[:,None].reshape(1,-1)[0]

array([43, 43, 49, 49, 49, 49, 49, 49, 43, 49, 49, 43, 49, 49,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3])

In [42]:
tokenizer_en.decode(target_tensor[6])

'that ninety nine percent of ours is newspaper and uh you know uh glass'

In [43]:
tokenizer_en.decode(out.argmax(axis = 1)[:,None].reshape(1,-1)[0])

'all all think think think think think think all think think all think think to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to '